In [2]:
import pandas as pd
import os

from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

import nltk
from nltk.corpus import stopwords
from nltk.stem import *


In [3]:
path = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/spam-filtering"

filenames = os.listdir(path)
videos = [pd.read_csv(path + '/' + filename, sep=',') for filename in filenames]

dataset = pd.concat(videos)

In [4]:
dataset.sample(frac=1).head(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
181,LneaDw26bFv1OK5quUOMAxJiN8tdhKPRzQMpB13UZpY,Jerry White,NaN,You guys should check out this EXTRAORDINARY w...,1
333,z13kjhwrdtvjzbbtc22ag3u4nmaocb023,Jamie Breedlove,2014-11-10T18:18:04,I love your music ﻿,0
292,z13wwxhgmr2ocvbks22mdpdwdu3lz3mze04,Jarelys Mellado,2015-02-23T00:37:24.817000,Check out this video on YouTube:<br /><br />﻿,1
81,z12jud4o3n34v5wph04ce3db0y2njn55gog0k,Kaya Roberts,2014-11-02T20:00:37,Admit it you just came here to check the numbe...,0
316,z12isnmj3wuovj4tt22stv0x1kblzls1w,Sandy Torres,2015-01-30T16:48:13.985000,Check out this playlist on YouTube:﻿,1
222,z13nujhysyf3ftvgt04ccrd4szrxcvzbvt00k,MeSoHornyMeLuvULongTime,2014-10-18T04:58:21,This video is so racist!!! There are only anim...,0
338,z13jzn1hvk35dxxzu04cj5lqmzv2thoquag,Sophie Flores,2015-01-10T18:17:21.145000,Check out this video on YouTube:﻿,1
5,z133yfmjdur4dvyjr04ceh2osl2fvngrqi4,Darrion Johnson,2015-05-29T01:27:30.360000,Wow this video almost has a billion views! Did...,0
352,z13isrygru30wbjmy23refqxdzizyp3xy04,Ala Fatah,2015-05-21T18:00:06.674000,Is that Megan Fox?﻿,0
145,z13byjoiuqvhctxc423ahhxafrizwhfto,tonya parker,2014-09-21T23:01:03,This is the best of the best video in world!!!...,0


In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
stop_words = stopwords.words("english")
vectorizer = CountVectorizer(stop_words=stop_words)

In [7]:
Y = dataset["CLASS"].tolist()

In [8]:
wnl = WordNetLemmatizer()

In [9]:
def hypertune_svm(x, y):
    model = svm.SVC()
    kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    C = [1000, 100, 50, 10, 1.0, 0.1, 0.01]
    gamma = ['scale']
    degree = range(1,11)
    # define grid search
    grid = dict(kernel=kernel,C=C,gamma=gamma, degree=degree)
    
    # model = LogisticRegression()
    # # solvers = ['newton-cg', 'lbfgs', 'liblinear']
    # solvers = ['saga', 'liblinear']
    # penalty = ['l1', 'l2']
    # c_values = [100, 10, 1.0, 0.1, 0.01]
    # # define grid search
    # grid = dict(solver=solvers, penalty=penalty, C=c_values)
    
    cv = KFold(n_splits=10)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='precision', error_score=0)
    grid_result = grid_search.fit(x, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [10]:
X_lem = vectorizer.fit_transform(dataset["CONTENT"].apply(wnl.lemmatize))
hypertune_svm(X_lem,Y)


KeyboardInterrupt



## Bags of words with stemming

In [11]:
stemmer = PorterStemmer()

In [ ]:
X_stem = vectorizer.fit_transform(dataset["CONTENT"].apply(stemmer.stem))
hypertune_svm(X_stem,Y)

In [12]:
model = svm.SVC(kernel='rbf', C=10, gamma='scale', degree=2)
scores_acc = cross_val_score(model, X_lem, Y, cv=10, scoring='accuracy')
scores_prec = cross_val_score(model, X_lem, Y, cv=10, scoring='precision')

In [13]:
scores_acc.mean(), scores_prec.mean()

(0.9488984824699109, 0.950040593728599)

In [14]:
path_to_results = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/researches/spam-filtering.tsv"
results = pd.read_csv(path_to_results, sep='\t')

results

,technique,kernel,C,gamma,degree,precision
0,lemmatization,linear,1000.00,scale,-,0.94113
1,lemmatization,poly,1000.00,scale,1,0.94113
2,lemmatization,rbf,1000.00,scale,-,0.93919
3,lemmatization,sigmoid,1000.00,scale,-,0.88844
4,lemmatization,linear,1000.00,scale,-,0.94113
...,...,...,...,...,...,...
555,stemming,sigmoid,0.01,scale,-,0.61341
556,stemming,linear,0.01,scale,-,0.98234
557,stemming,poly,0.01,scale,10,0.95500
558,stemming,rbf,0.01,scale,-,0.73248
